# NOT-Example 
## Finally some Code!

In [1]:
import qiskit
from qiskit.providers import BaseBackend
from qiskit.providers.ibmq import IBMQBackend

import os
qiskit.IBMQ.enable_account(os.getenv("IBMQX_TOKEN"))
ibmq5: BaseBackend = qiskit.IBMQ.get_backend('ibmq_16_melbourne')

C:\Users\cncen\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
import numpy as np

X_train = np.asarray([[1.0, 1.0], [-1.0, 1.0], [-1.0, -1.0], [1.0, -1.0]])
y_train = [0, 1, 0, 1]

X_test = np.asarray([[0.2, 0.4], [0.4, -0.8]])
y_test = [0, 1]

from dc_qiskit_qml.feature_maps import FeatureMap
import scipy.sparse as sparse


class MyFeatureMap(FeatureMap):
        def map(self, input_vector: list) -> sparse.dok_matrix:
            result = sparse.dok_matrix((4,1))
            index = 0
            if input_vector[0] > 0 and input_vector[1] > 0:
                index = 0
            if input_vector[0] < 0 < input_vector[1]:
                index = 1
            if input_vector[0] < 0 and input_vector[1] < 0:
                index = 2
            if input_vector[0] > 0 > input_vector[1]:
                index = 3
            result[index, 0] = 1.0
            return result


feature_map = MyFeatureMap()

ModuleNotFoundError: No module named 'dc_qiskit_qml'

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(
    X_train[:,0], X_train[:,1], 
    color=['red' if yy == 0 else 'blue' for yy in y_train],
    marker='x', s=200)
plt.scatter(
    X_test[:,0], X_test[:,1], 
    color=['red' if yy == 0 else 'blue' for yy in y_test],
    marker='o', s=200)
plt.hlines(0.0, -1.0, 1.0)
plt.vlines(0.0, -1.0, 1.0)
plt.show()

In [ ]:
from dc_qiskit_qml.distance_based.hadamard import QmlHadamardNeighborClassifier
from dc_qiskit_qml.distance_based.hadamard.state import QmlBinaryDataStateCircuitBuilder
from dc_qiskit_qml.distance_based.hadamard.state.cnot import CCXToffoli

initial_state_builder = QmlBinaryDataStateCircuitBuilder(CCXToffoli(), do_optimizations=False)

execution_backend: BaseBackend = qiskit.Aer.get_backend('qasm_simulator')
qml = QmlHadamardNeighborClassifier(backend=execution_backend,
                                    shots=8192,
                                    feature_map=feature_map,
                                    classifier_circuit_factory=initial_state_builder)

qml.fit(X_train, y_train)
qml.predict(X_test), y_test

In [ ]:
qml._last_predict_circuits[0].draw(output='mpl')

In [ ]:
initial_state_builder = QmlBinaryDataStateCircuitBuilder(CCXToffoli(), do_optimizations=True)

execution_backend: BaseBackend = qiskit.Aer.get_backend('qasm_simulator')
qml = QmlHadamardNeighborClassifier(backend=execution_backend,
                                    shots=8192,
                                    feature_map=feature_map,
                                    classifier_circuit_factory=initial_state_builder)

qml.fit(X_train, y_train)
qml.predict(X_test), y_test

In [ ]:
qml._last_predict_circuits[0].draw(output='mpl')

In [ ]:
qobj = qiskit.compile(qml._last_predict_circuits[0], backend=ibmq5)

In [3]:
from qiskit.converters import circuit_to_dag, dag_to_circuit, qobj_to_circuits
print("Number of gates: {}.\n".format(len(qobj_to_circuits(qobj)[0].data)))
# print(qobj_to_circuits(qobj)[0].qasm())
qobj_to_circuits(qobj)[0].draw(line_length=120)

NameError: name 'qobj' is not defined